**Phase 1: Implement Basic NLP Tasks (POS, Embedding Techniques)**

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

def set_embedding_model():
    return HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [6]:
import os
import shutil
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain_chroma import Chroma

In [4]:
data_path = r"C:\PES\CSSEM-6\GenAI\ChessBot\data"
database_path = "database"

In [5]:
def set_database():
    database = Chroma(
        database_path,
        embedding_function=set_embedding_model(),
        persist_directory=database_path,
    )
    return database

def delete_database():
    if os.path.exists(database_path):
        shutil.rmtree(database_path)
        print("Database deleted")
    else:
        print("Database does not exist")


In [8]:
def load_documents():
    document_loader = PyPDFDirectoryLoader(data_path)
    return document_loader.load()

documents = load_documents()
print(f"Loaded {len(documents)} documents.")


Loaded 1263 documents.


In [9]:
def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=20,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

chunks = split_text(documents)
print(f"Split into {len(chunks)} chunks.")

Split into 1731 chunks.


In [10]:
def calculate_chunk_ids(chunks):
    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        chunk.metadata["id"] = chunk_id

    return chunks

chunks_with_ids = calculate_chunk_ids(chunks)


In [11]:
def create_database(chunks: list[Document]):
    database = set_database()
    chunks_with_ids = calculate_chunk_ids(chunks)

    existing_items = database.get(include=[])
    existing_ids = set(existing_items["ids"])
    print(f"Existing documents in database: {len(existing_ids)}")

    new_chunks = [chunk for chunk in chunks_with_ids if chunk.metadata["id"] not in existing_ids]

    if len(new_chunks):
        print(f"Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        database.add_documents(new_chunks, ids=new_chunk_ids)
        print("New documents added successfully.")
    else:
        print("No new documents to add.")

create_database(chunks_with_ids)

C:\Users\krish\AppData\Local\Temp\ipykernel_9208\1165023105.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")



Existing documents in database: 0
Adding new documents: 1731
New documents added successfully.
